<a href="https://colab.research.google.com/github/dak-sh-kim/selfstudy-wikidocs-tensorflow-nlp-tutorial-notebooks/blob/main/20220612_NLP_2_06)_Integer_Encoding_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 06) 정수 인코딩 (Integer Encoding)

- 컴퓨터가 숫자를 잘 처리하기 때문에, NLP에는 텍스트 --> 숫자 바꾸는 여러가지 기법
- 첫 단계: 맵핑(mapping) - 각 단어를 고유한 정수에 맵핑 
- ex. 5000개의 단어를 갖고 있다면, 각 단어에 인덱스를 부여하는 것 - book-221번, animal - 12번, etc.
- 인덱스를 부여하는 방법: 여러가지, 보통 단어 등장 빈도수 기준 정렬

## 1. 정수 인코딩(Integer Encoding)

- 정수 인코딩의 목적은 원-핫 인코딩 실습/워드 임베딩 챕터에서 알아보자
- 여기서는 정수 인코딩의 과정만 정리

#### 1) dictionary 사용하기

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
raw_text = """A barber is a person. a barber is good person. a barber is huge person. 
he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. 
a barber kept his word. His barber kept his secret.
But keeping and keeping such a huge secret to himself was driving the barber crazy. 
the barber went up a huge mountain."""

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# 문장 토큰화
sentences = sent_tokenize(raw_text)
print(sentences)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

+ 텍스트를 수치화하는 단계라는 것은 본격적으로 자연어처리 작업에 들어간다는 의미
+ 따라서, 단어가 텍스트일 때만 할 수 있는 최대한의 전처리를 끝내놓아야 함

In [ ]:
vocab = {}
preprocessed_sentences = []
stop_words = set(stopwords.words('english'))

for sentence in sentences:
  # 단어 토큰화
  tokenized_sentence = word_tokenize(sentence)
  result = []

  for word in tokenized_sentence:
    word = word.lower() # 모든 단어를 '소문자화' 하여 단어의 개수를 줄인다.
    if word not in stop_words: # 1) 단어 토큰화 된 결과에 대해 '불용어' 제거
      if len(word) > 2: # 2) '단어길이 2 이하' 추가 제거

        result.append(word)
        if word not in vocab:
          vocab[word] = 0
        vocab[word] += 1 # a += b : a = a+b ; a += 1 : a = a+1
        # vocab; word 초기값 = 0
          # word1: 빈도수(생길때마다 증가), word2: 빈도수, word3:빈도수... 
  preprocessed_sentences.append(result)
print(preprocessed_sentences)      

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [ ]:
print("단어 집합: ", vocab)

단어 집합:  {'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


- vocab은 {딕셔너리 구조; key: value}

In [ ]:
vocab['barber']

8

In [ ]:
# 빈도수가 높은 순으로 정렬
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse = True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


- 높은 빈도수를 가진 단어일수록 낮은 정수를 부여, 정수는 1부터 부여

In [ ]:
word_to_index = {}
i = 0

for (word, frequency) in vocab_sorted:
  if frequency > 1 : # 빈도수가 작은 단어는 제외 - 등장빈도가 적으면 자연어처리에서는 의미 가지지 X 가능성 높음
    i = i + 1 # 그 다음 순서
    word_to_index[word] = i # 순서 할당

print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


- 빈도수가 높은 n개의 단어만 사용하고 싶다? 

In [ ]:
vocab_size = 5

# 인덱스가 5 초과인 단어 제거
words_frequency = [word for word, index in word_to_index.items() if index >= vocab_size + 1]

# 해당 단어에 대한 인덱스 정보를 삭제
for w in words_frequency:
  del word_to_index[w]
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


- Out-Of-Vocabulary (단어 집합에 없는 단어가 생기는 상황) 문제

In [ ]:
# 단어 집합에 없는 단어들을 'OOV' 인덱스로 인코딩
word_to_index['OOV'] = len(word_to_index) + 1 
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}


- word_to_index를 사용하여 sentences의 모든 단어들을 맵핑되는 정수로 인코딩

In [ ]:
encoded_sentences = []
for sentence in preprocessed_sentences:
  encoded_sentence = []
  for word in sentence:
    try:
      # 단어 집합에 있는 단어라면 해당 단어의 정수를 리턴.
      encoded_sentence.append(word_to_index[word])
    except KeyError:
      # 만약 단어집합에 없는 단어라면 'OOV'의 정수를 리턴.
      encoded_sentence.append(word_to_index['OOV'])
  encoded_sentences.append(encoded_sentence)
print(encoded_sentences)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


- 지금까지는 dictionary 자료형으로 정수 인코딩을 진행
- 더 쉽게 하기 위해서: Counter, FreqDist, enumerate 사용 or 케라스 토크나이저 사용 추천

#### 2) Counter 사용하기

In [ ]:
from collections import Counter

In [ ]:
preprocessed_sentences
# 단어 토큰화가 된 결과임 

[['barber', 'person'],
 ['barber', 'good', 'person'],
 ['barber', 'huge', 'person'],
 ['knew', 'secret'],
 ['secret', 'kept', 'huge', 'secret'],
 ['huge', 'secret'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'secret'],
 ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
 ['barber', 'went', 'huge', 'mountain']]

- 하나의 단어집합 (vocabulary) 만들어보자
- **문장의 경계인 [, ] 제거, 단어들을 하나의 리스트 덩어리로 만들기

In [ ]:
# words = np.hstack(preprocessed_sentences)으로도 수행 가능.

all_words_list = sum(preprocessed_sentences, [])
print(all_words_list)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


- 중복을 제거하고 단어의 빈도수를 기록

In [ ]:
# 파이썬의 Counter모듈을 이용하여 단어의 빈도수 카운트
vocab = Counter(all_words_list)
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


In [ ]:
vocab['barber'] # 단어를 key로, 단어에 대한 빈도수를 value로 저장, 단어를 입력하면 빈도수 리턴

8

- most_common()은 상위 빈도수를 가진 주어진 수의 단어만을 리턴
- 등장 빈도수가 높은 단어들을 원하는 개수만큼만 얻을 수 있음
- 등장 빈도수 상위 5개의 단어만 단어집합으로 저장할 수 있음

In [ ]:
vocab_size = 5
vocab = vocab.most_common(vocab_size) # 등장 빈도가 높은 상위 5개의 단어만 저장
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

- 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스 부여

In [ ]:
word_to_index = {}
i = 0

for (word, frequency) in vocab:
  i = i + 1
  word_to_index[word] = i

print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


#### 3) nltk의 FreqDist사용하기

In [ ]:
from nltk import FreqDist
import numpy as np

In [ ]:
# np.hstack으로 문장 구분을 제거 + 빈도수 계산
print(preprocessed_sentences)
vocab = FreqDist(np.hstack(preprocessed_sentences))
vocab

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


FreqDist({'barber': 8,
          'crazy': 1,
          'driving': 1,
          'good': 1,
          'huge': 5,
          'keeping': 2,
          'kept': 4,
          'knew': 1,
          'mountain': 1,
          'person': 3,
          'secret': 6,
          'went': 1,
          'word': 2})

In [ ]:
# 단어를 key로, 단어에 대한 빈도수를 value로 저장, 단어를 입력하면 빈도수 리턴
print(vocab['barber'])

8


In [ ]:
vocab_size = 5
vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 5개의 단어만 저장
print(vocab)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]


- 인덱스 부여: 앞에서와 달리 enumerate()사용하여 좀 더 짧은 코드로 부여

In [ ]:
word_to_index = {word[0]: index + 1 for index, word in enumerate(vocab)}
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


#### 4) enumerate 이해하기

- enumerate()는 순서가 있는 자료형(list, set, tuple, dictionary, string)을 입력으로 받아 인덱스를 순차적으로 함께 리턴

In [ ]:
test_input = ['a', 'b', 'c', 'd', 'e']
for index, value in enumerate(test_input):
  print("value:{}, index:{}".format(value, index)) # 인덱스의 모든 토큰에 대해 인덱스가 순차적으로 증가되며 부여

value:a, index:0
value:b, index:1
value:c, index:2
value:d, index:3
value:e, index:4


## 2. 케라스(Keras)의 텍스트 전처리

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
print(preprocessed_sentences) # 단어 토큰화까지 수행된 앞의 데이터 사용
  # 참고: 1) character 2이하 삭제 2) stopword 삭제 3) 소문자 변환

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [ ]:
tokenizer = Tokenizer()

# fit_on_texts()안에 코퍼스를 입력으로 하면, 빈도수를 기준으로 단어 집합 생성
tokenizer.fit_on_texts(preprocessed_sentences)

- fit_on_text()는 입력한 텍스트로부터 단어 빈도수가 높은 순으로 1, 2, 3 정수인덱스 부여
- 정확히 앞에서 했던 정수 인코딩 작업이 이루어짐
- 각 단어에 인덱스가 어떻게 부여되었는지 보려면, word_index 사용

In [ ]:
print(tokenizer.word_index) # 빈도 높은 순

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


- 각 단어의 카운트 수를 보려면, word_counts 사용

In [ ]:
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


- text_to_sequences()는 입력으로 들어온 코퍼스에 대해 각 단어를 이미 정해진 인덱스로 변환

In [ ]:
print(tokenizer.texts_to_sequences(preprocessed_sentences))

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


- cf. 앞서 빈도수 가장 높은 단어 n개: most_common()사용
- tokenizer = Tokenizer(num_words = 숫자) 와 같은 방법으로 빈도수 가장 높은 단어 사용 지정
- 여기서는 1번부터 5번까지 단어만 사용 (상위 5개만 사용)

In [ ]:
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size+1) # 상위 5개 단어만 사용
  # num_words + 1하는 이유: num_words는 숫자를 0부터 카운트 하기 때문에, 만약 5를 넣으면 0~4 단어보존 의미
    # 뒤의 실습에서 1번 단어부터 4번 단어만 남게 됨. 따라서 1~5까지 사용하고 싶다면 5+1 넣어야 함
    # 실질적으로 숫자 0에 지정된 단어가 존재하지 않는데도, 케라스 토크나이저가 숫자 0까지 단어집합 크기로 산정하는 이유는?
      # 자연어처리에서의 패딩 작업 때문임. 
tokenizer.fit_on_texts(preprocessed_sentences)

- 참고) word_index와 word_counts에서도 지정된 num_words만큼의 단어만 남기고 싶다면? (굳이 필요 없긴 함)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_sentences)

In [ ]:
vocab_size = 5
words_frequency = [word for word, index in tokenizer.word_index.items() if index >= vocab_size+1]

# 인덱스가 5 초과된 단어 제거
for word in words_frequency:
  del tokenizer.word_index[word]
  del tokenizer.word_counts[word]

print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(preprocessed_sentences))

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}
OrderedDict([('barber', 8), ('person', 3), ('huge', 5), ('secret', 6), ('kept', 4)])
[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


- 케라스 토크나이저는 OOV(Out-of-Vocab:단어집합에 없는 단어) 에 대해서는 단어를 정수로 바꾸는 과정에서 아예 단어 제거
- 단어 집합에 없는 단어들은 OOV로 간주하여 보존하고 싶다면, Tokenizer의 인자 oov_token을 사용

In [ ]:
# 숫자 0과 OOV를 고려해서, 단어 집합의 크기는 +2
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size +2, oov_token = 'OOV')
tokenizer.fit_on_texts(preprocessed_sentences)

- 만약 oov_token을 사용하기로 했다면, 케라스 토크나이저는 기본적으로 'OOV'의 인덱스를 1로 함

In [ ]:
print('단어 OOV의 인덱스 :{}'.format(tokenizer.word_index['OOV']))

단어 OOV의 인덱스 :1


- 이제 코퍼스에 대해서 정수 인코딩을 진행

In [ ]:
print(tokenizer.texts_to_sequences(preprocessed_sentences))
  # good은 빈도수가 상위 9이므로, 앞에서 vocab_size에 의해 걸러졌기 때문에 OOV:1로 인코딩됨
  # 빈도수 상위 5개의 단어는 2~6까지의 인덱스를 가졌고, 그 외 단어들은 모두 OOV:1로 치환

[[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]


- **good이 OOV가 된 과정 (하단)

In [ ]:
preprocessed_sentences

[['barber', 'person'],
 ['barber', 'good', 'person'],
 ['barber', 'huge', 'person'],
 ['knew', 'secret'],
 ['secret', 'kept', 'huge', 'secret'],
 ['huge', 'secret'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'secret'],
 ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
 ['barber', 'went', 'huge', 'mountain']]

In [ ]:
tokenizer.word_index

{'OOV': 1,
 'barber': 2,
 'crazy': 12,
 'driving': 11,
 'good': 9,
 'huge': 4,
 'keeping': 8,
 'kept': 5,
 'knew': 10,
 'mountain': 14,
 'person': 6,
 'secret': 3,
 'went': 13,
 'word': 7}